Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 3.86 ms, sys: 563 µs, total: 4.42 ms
Wall time: 12.9 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

<Client: 'inproc://10.1.0.4/7455/1' processes=1 threads=6, memory=7.29 GB>

1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 211 ms, sys: 2.11 ms, total: 213 ms
Wall time: 1.83 s


[1.7933077020435082,
 1.8089798136803643,
 1.848088017108008,
 1.2239674097087225,
 1.0868082270427575,
 1.6412189836147357,
 1.2901174632481,
 1.0278335045391547,
 1.3082981931341273,
 1.934408159375384,
 1.6831074661594898,
 1.4585931720659442,
 1.2031503171698819,
 1.2434295014765797,
 1.5123729496447083,
 1.8002337591563888,
 1.7020215007751989,
 1.9965960924693973,
 1.4128252313592375,
 1.5807415198551973,
 1.5631942524401214,
 1.5362980571044722,
 1.565354602519641,
 1.4678080085608693,
 1.926311560564486,
 1.1120157857887225,
 1.8860811531995478,
 1.2942735581774916,
 1.1152113679457547,
 1.6431322466646665,
 1.0786460214267555,
 1.9320397571277748,
 1.3216208145772756,
 1.0912227905211211,
 1.3893335420279478,
 1.8548868430141483,
 1.656940713045493,
 1.5120580026460946,
 1.3690620143580137,
 1.0591747917545686,
 1.7419159346203248,
 1.2037063819856608]

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.5619205697581018,
 1.6238706568683139,
 1.76674575055971,
 1.7850484891082403,
 1.8885212544055472,
 1.9380935073899481,
 1.3996858259259755,
 1.0878280991450553,
 1.439174333437816,
 1.8708863815446444,
 1.2510840644531702,
 1.490719410996884,
 1.8077315148702247,
 1.3674478268577768,
 1.9187517421249032,
 1.3386315627601073,
 1.9672705781700472,
 1.2689445605166252,
 1.8335780900300294,
 1.8392097996752925,
 1.3047312100907296,
 1.2154993395714624,
 1.7671425125956781,
 1.147144316285813,
 1.6530944885280414,
 1.9240616225995306,
 1.826079964527184,
 1.9603630547319881,
 1.3691211483841612,
 1.6308404418820093,
 1.894506093675203,
 1.1544183389251204,
 1.2394099918144934,
 1.461758233844785,
 1.810542075356682,
 1.001920266675211,
 1.1893667293121237,
 1.0365895017584168,
 1.9624380934040708,
 1.7990312982700198,
 1.7976483864399664,
 1.6710831081084359,
 1.732064162852522,
 1.708744622971671,
 1.6300732385771948,
 1.736436925758387,
 1.5638065750237096,
 1.3346540969228138,
 1.80

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 194 ms, sys: 23.6 ms, total: 218 ms
Wall time: 2.17 s
